# Домашнє завдання до теми "Розширені методи обробки даних у Pandas"

В цьому домашньому завданні продовжуємо працювати з набором даних `supermarket_sales.csv`.

0. Імпортуйте бібліотеку pandas та зчитайте дані у змінну `df` типу `pandas.DataFrame`.

In [3]:
import pandas as pd

In [4]:
data_path = '../content/drive/MyDrive/supermarket_sales.csv'
df = pd.read_csv(data_path)

In [5]:
type(df)

pandas.core.frame.DataFrame

1. Відсортуйте дані за спаданням ціни товара ('Unit price') і виведіть 5 перших рядочків на екран.

In [6]:
df.sort_values(by = "Unit price", ascending=False).head(5)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
983,148-41-7930,C,Naypyitaw,Normal,Male,Health and beauty,99.96,7,34.986,734.706,1/23/2019,10:33,Cash,699.72,4.761905,34.986,6.1
122,219-22-9386,B,Mandalay,Member,Male,Sports and travel,99.96,9,44.982,944.622,3/9/2019,17:26,Credit card,899.64,4.761905,44.982,4.2
930,641-62-7288,B,Mandalay,Normal,Male,Home and lifestyle,99.92,6,29.976,629.496,3/24/2019,13:33,Ewallet,599.52,4.761905,29.976,7.1
494,437-53-3084,B,Mandalay,Normal,Male,Fashion accessories,99.89,2,9.989,209.769,2/26/2019,11:48,Ewallet,199.78,4.761905,9.989,7.1
283,667-92-0055,A,Yangon,Member,Male,Health and beauty,99.83,6,29.949,628.929,3/4/2019,15:02,Ewallet,598.98,4.761905,29.949,8.5


2. Знайдіть інвойс (рядок в даних) в категорії товарів `Electronic accessories`, в якому був куплений товар з найбільшою кількістю і найбільшою ціною. Виведіть цей рядок даних на екран.

In [7]:
number_one = (
    df[df['Product line'] == 'Electronic accessories']
.sort_values(['Unit price', 'Quantity'],ascending = False)
.head(1)
)
number_one

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
209,817-69-8206,B,Mandalay,Normal,Female,Electronic accessories,99.73,9,44.8785,942.4485,3/2/2019,19:42,Credit card,897.57,4.761905,44.8785,6.5


3.1. Створіть нову колонку в даних `Rating Category`, яка визначає категорію рейтингу (`Rating`): високий, середній чи низький за наступним правилом:
- якщо рейтинг вищий або рівний 8 - то він високий (`High`)
- якщо рейтинг від 6 (включно) до 8 (не включно) - то він середній (`Medium`)
- інакше - низький (`Low`).

In [8]:
def category_mapping(Rating):
    if Rating >= 8:
        return 'High'
    elif Rating >= 6 < 8:
        return 'Medium'
    else:
        return 'Low'

In [9]:
df['Rating_Category'] = df.Rating.apply(category_mapping)
df['Rating_Category']

,Rating_Category
0,High
1,High
2,Medium
3,High
4,Low
...,...
995,Medium
996,Low
997,Medium
998,Low


3.2. Виведіть, який відсоток даних становить кожне зі значень категорій рейтингу.

In [10]:
percent_rating_category = (
    df['Rating_Category']
    .value_counts(normalize=True) * 100

)
percent_rating_category

,proportion
Rating_Category,
Medium,35.6
High,32.9
Low,31.5


4.1. Запустіть код нижче аби створити змінну `time`. Ця змінна містить час в форматі "година:хвилина". Напишіть функцію `get_hour`, яка буде зі змінної формату як `time` дістати лише години і конвертувати їх у тип `int`.

Запустіть написану функцію на змінній `time` та перевірте правильність роботи функції.

In [11]:
time = '11:28'
type(time)

str

In [12]:
def get_hour(time: str) -> int:
  return pd.to_datetime(time, format= "%H:%M").hour


In [13]:
get_hour(time)

11

4.2. З допомогою функції `get_hour` і метода `DataFrame.apply()` створіть нову колонку `Hour` в нашому наборі даних, яка буде містити години здійснення покупки, обчислені на основі часу здійснення покупки `Time`.

In [14]:
df['Hour'] = df.Time.apply(get_hour)
df['Hour']

,Hour
0,13
1,10
2,13
3,20
4,10
...,...
995,13
996,17
997,13
998,15


4.3. Виведіть кількість значень за кожною годиною покупки і дайте відповідь "О яких годинах найбільше здійснюють покупок покупці і чому на Ваш погляд саме о цих годинах? Чи це слогічно, що найбільше покупок саме в ці години?".

In [15]:
df.groupby(by=['Hour'])['Hour'].count()

,Hour
Hour,
10,101
11,90
12,89
13,103
14,83
15,102
16,77
17,74
18,93


Найбільше покупок здійснюється о 10, 13, 15 та 19 годинах. Для мене є логічними та зрозумілим , що покупці купують в ранковий час 10, обідній 13 та вечірній 19. Це повязано з більш сталим робочим днем , можливо - до роботи, на обідній перерві та після роботи.
15 година стала активним трафіком, за рахунок школярів, студентів, безробітніх та людей, що працють за не класичними робочим графіком.

5. Виведіть середнє значення чеку (`Total`) для покупок в містах, наявних в даних, за допомогою функції `groupby`.

In [16]:
df.groupby(by = ['City'])['Total'].mean().round(3)

,Total
City,
Mandalay,319.873
Naypyitaw,337.100
Yangon,312.354


6. Виведіть кількість покупок, середнє, мінімальне та максимальне значення для загальної суми чеку (Total) в містах, наявних в даних, за допомогою функції groupby.
Зробіть висновки з цих показників та дайте рекомендації для мережі супермаркетів.

In [17]:
df.groupby('City').agg({
   'Total' : ['mean', 'min', 'max'],
   'Quantity': ['count']
}).round(3)

Total                  Quantity
              mean     min      max    count
City                                        
Mandalay   319.873  18.638  1022.49      332
Naypyitaw  337.100  10.678  1042.65      328
Yangon     312.354  12.694  1039.29      340

Mandalay - має досить не погані показники. Найбільша мінімальна ціна, та при цьому нейменша макимальна. За кільіксітю продажів на другому місці серед міст. Поради:
1. Спостерігати, щоб товари , які продаються найбільше - були в наявності.
2. Додавати акції та спец. пропозиції для продажу нашої преміум продукції.

Naypyitaw	- можемо помітити найменшу кількість продажів, але найбільшу максимальну та середню ціну. Можу порекомендувати цьому магазину кілька варіантів:
1. Пошук постачальників з нищою закупівельною ціною, щоб продавати більше, але заробляти на оберті продажів.
2. Робити акції та особливі пропозиції, щоб знову ж таки збільшити кількість продажів.
3. Вивести товар, що продається найменше з асортименту. Залишити те, що дійсно продається.
4. Фокус на найдорожчі товари(преміум), підключити маркетинг, щоб вони дійсно успішно продавались.

Yangon -  в цьому місті є набільша кількість продажів. При цьому середня ціна серед міст - найменша. Можемо зробити висновок, що це відбувається
за рахунок оберту продажів. Поради:
1. Слідкувати , щоб товари , що створюють оберт - були в наявності.
2. Просувати преміум товари також. Почати зі знижки, щоб покупуць познайомився з продуктом.

7. Ми внесли деякі зміни в наш набір даних, тож, давайте запишемо результат у файл. Запишіть оновлену версію даних у файл `supermarket_sales_enriched.csv`. Зверніть увагу, аби не записати нічого зайвого, наприклад, індекси рядків нам в файлі не потрібні, оскільки вони стандартні.

In [18]:
df.to_csv('supermarket_sales_enriched.csv', index=False)

In [19]:
pd.read_csv('supermarket_sales_enriched.csv', nrows=5)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating,Rating_Category,Hour
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1,High,13
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6,High,10
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4,Medium,13
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4,High,20
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3,Low,10
